In [1]:
%load_ext Cython

In [2]:
%%cython
#cython: cdivision=True
#cython: boundscheck=False
#cython: nonecheck=False
#cython: wraparound=False

import numpy as np
from scipy import ndimage as ndi

cimport numpy as cnp
from libc.string cimport memcpy
cnp.import_array()

def _table_lookup_3d(image, table):
    indexer = _table_lookup_index_3d(np.ascontiguousarray(image, np.uint8))
    image = table[indexer]
    return image

def _table_lookup_index_3d(cnp.uint8_t[:, :, ::1] image):
    cdef:
        Py_ssize_t[:, :, ::1] indexer
        Py_ssize_t *p_indexer
        cnp.uint8_t *p_image
        Py_ssize_t i_stride
        Py_ssize_t j_stride
        Py_ssize_t i_shape
        Py_ssize_t j_shape
        Py_ssize_t k_shape
        Py_ssize_t i
        Py_ssize_t j
        Py_ssize_t k
        Py_ssize_t ii
        Py_ssize_t jj
        Py_ssize_t kk
        Py_ssize_t offset
        Py_ssize_t[:, :, :] structure_3d
    structure_3d = np.arange(27, dtype=np.intp).reshape(3,3,3)
    structure_3d = pow(np.intp(2),structure_3d)
    i_shape   = image.shape[0]
    j_shape   = image.shape[1]
    k_shape   = image.shape[2]
    indexer = np.zeros((i_shape, j_shape, k_shape), dtype=np.intp)
    p_indexer = &indexer[0, 0, 0]
    p_image   = &image[0, 0, 0]
    i_stride  = image.strides[0]
    j_stride  = image.strides[1]
    assert i_shape >= 3 and j_shape >= 3, \
        "arrays has to be > 3x3, for < 3x3 method you can look into the function _table_lookup in skimage source code"
    with nogil:
        for i in range(1, i_shape-1):
            for j in range(1, j_shape - 1):
                offset = i_stride * i + j_stride * j + 1
                for k in range(1, k_shape - 1):
                    if p_image[offset]:
                        p_indexer[offset + i_stride + j_stride + 1] += structure_3d[0, 0, 0]
                        p_indexer[offset + i_stride + j_stride] += structure_3d[0, 0, 1]
                        p_indexer[offset + i_stride + j_stride - 1] += structure_3d[0, 0, 2]

                        p_indexer[offset + i_stride + 1] += structure_3d[0, 1, 0]
                        p_indexer[offset + i_stride] += structure_3d[0, 1, 1]
                        p_indexer[offset + i_stride - 1] += structure_3d[0, 1, 2]
                        
                        p_indexer[offset + i_stride - j_stride + 1] += structure_3d[0, 2, 0]
                        p_indexer[offset + i_stride - j_stride] += structure_3d[0, 2, 1]
                        p_indexer[offset + i_stride - j_stride - 1] += structure_3d[0, 2, 2]
                        
                        p_indexer[offset + j_stride + 1] += structure_3d[1, 0, 0]
                        p_indexer[offset + j_stride] += structure_3d[1, 0, 1]
                        p_indexer[offset + j_stride - 1] += structure_3d[1, 0, 2]
                        
                        p_indexer[offset + 1] += structure_3d[1, 1, 0]
                        p_indexer[offset] += structure_3d[1, 1, 1]
                        p_indexer[offset - 1] += structure_3d[1, 1, 2]
                        
                        p_indexer[offset - j_stride + 1] += structure_3d[1, 2, 0]
                        p_indexer[offset - j_stride] += structure_3d[1, 2, 1]
                        p_indexer[offset - j_stride - 1] += structure_3d[1, 2, 2]
                        
                        p_indexer[offset - i_stride + j_stride + 1] += structure_3d[2, 0, 0]
                        p_indexer[offset - i_stride + j_stride] += structure_3d[2, 0, 1]
                        p_indexer[offset - i_stride + j_stride - 1] += structure_3d[2, 0, 2]
                        
                        p_indexer[offset - i_stride + 1] += structure_3d[2, 1, 0]
                        p_indexer[offset - i_stride] += structure_3d[2, 1, 1]
                        p_indexer[offset - i_stride - 1] += structure_3d[2, 1, 2]
                        
                        p_indexer[offset - i_stride - j_stride + 1] += structure_3d[2, 2, 0]
                        p_indexer[offset - i_stride - j_stride] += structure_3d[2, 2, 1]
                        p_indexer[offset - i_stride - j_stride - 1] += structure_3d[2, 2, 2]
                        
                    offset += 1
        #
        # Do the corner cases (literally)
        #
        if image[0, 0, 0]:
            indexer[0, 0, 0] += structure_3d[1, 1, 1]
            indexer[0, 0, 1] += structure_3d[1, 1, 0]
            indexer[0, 1, 0] += structure_3d[1, 0, 1]
            indexer[0, 1, 1] += structure_3d[1, 0, 0]
            
            indexer[1, 0, 0] += structure_3d[0, 1, 1]
            indexer[1, 0, 1] += structure_3d[0, 1, 0]
            indexer[1, 1, 0] += structure_3d[0, 0, 1]
            indexer[1, 1, 1] += structure_3d[0, 0, 0]

        if image[0, 0, k_shape - 1]:
            indexer[0, 0, k_shape - 2] += structure_3d[1, 1, 2]
            indexer[0, 0, k_shape - 1] += structure_3d[1, 1, 1]
            indexer[0, 1, k_shape - 2] += structure_3d[1, 0, 2]
            indexer[0, 1, k_shape - 1] += structure_3d[1, 0, 1]
            
            indexer[1, 0, k_shape - 2] += structure_3d[0, 1, 2]
            indexer[1, 0, k_shape - 1] += structure_3d[0, 1, 1]
            indexer[1, 1, k_shape - 2] += structure_3d[0, 0, 2]
            indexer[1, 1, k_shape - 1] += structure_3d[0, 0, 1]
        
        if image[0, j_shape - 1, 0]:
            indexer[0, j_shape - 2, 0] += structure_3d[1, 2, 1]
            indexer[0, j_shape - 2, 1] += structure_3d[1, 2, 0]
            indexer[0, j_shape - 1, 0] += structure_3d[1, 1, 1]
            indexer[0, j_shape - 1, 1] += structure_3d[1, 1, 0]
            
            indexer[1, j_shape - 2, 0] += structure_3d[0, 2, 1]
            indexer[1, j_shape - 2, 1] += structure_3d[0, 2, 0]
            indexer[1, j_shape - 1, 0] += structure_3d[0, 1, 1]
            indexer[1, j_shape - 1, 1] += structure_3d[0, 1, 0]
            
        if image[0, j_shape - 1, k_shape - 1]:
            indexer[0, j_shape - 2, k_shape - 2] += structure_3d[1, 2, 2]
            indexer[0, j_shape - 2, k_shape - 1] += structure_3d[1, 2, 1]
            indexer[0, j_shape - 1, k_shape - 2] += structure_3d[1, 1, 2]
            indexer[0, j_shape - 1, k_shape - 1] += structure_3d[1, 1, 1]
            
            indexer[1, j_shape - 2, k_shape - 2] += structure_3d[0, 2, 2]
            indexer[1, j_shape - 2, k_shape - 1] += structure_3d[0, 2, 1]
            indexer[1, j_shape - 1, k_shape - 2] += structure_3d[0, 1, 2]
            indexer[1, j_shape - 1, k_shape - 1] += structure_3d[0, 1, 1]
            
        ####
        
        if image[i_shape - 1, 0, 0]:
            indexer[i_shape - 2, 0, 0] += structure_3d[2, 1, 1]
            indexer[i_shape - 2, 0, 1] += structure_3d[2, 1, 0]
            indexer[i_shape - 2, 1, 0] += structure_3d[2, 0, 1]
            indexer[i_shape - 2, 1, 1] += structure_3d[2, 0, 0]
            
            indexer[i_shape - 1, 0, 0] += structure_3d[1, 1, 1]
            indexer[i_shape - 1, 0, 1] += structure_3d[1, 1, 0]
            indexer[i_shape - 1, 1, 0] += structure_3d[1, 0, 1]
            indexer[i_shape - 1, 1, 1] += structure_3d[1, 0, 0]

        if image[i_shape - 1, 0, k_shape - 1]:
            indexer[i_shape - 2, 0, k_shape - 2] += structure_3d[2, 1, 2]
            indexer[i_shape - 2, 0, k_shape - 1] += structure_3d[2, 1, 1]
            indexer[i_shape - 2, 1, k_shape - 2] += structure_3d[2, 0, 2]
            indexer[i_shape - 2, 1, k_shape - 1] += structure_3d[2, 0, 1]
            
            indexer[i_shape - 1, 0, k_shape - 2] += structure_3d[1, 1, 2]
            indexer[i_shape - 1, 0, k_shape - 1] += structure_3d[1, 1, 1]
            indexer[i_shape - 1, 1, k_shape - 2] += structure_3d[1, 0, 2]
            indexer[i_shape - 1, 1, k_shape - 1] += structure_3d[1, 0, 1]
        
        if image[i_shape - 1, j_shape - 1, 0]:
            indexer[i_shape - 2, j_shape - 2, 0] += structure_3d[2, 2, 1]
            indexer[i_shape - 2, j_shape - 2, 1] += structure_3d[2, 2, 0]
            indexer[i_shape - 2, j_shape - 1, 0] += structure_3d[2, 1, 1]
            indexer[i_shape - 2, j_shape - 1, 1] += structure_3d[2, 1, 0]
            
            indexer[i_shape - 1, j_shape - 2, 0] += structure_3d[1, 2, 1]
            indexer[i_shape - 1, j_shape - 2, 1] += structure_3d[1, 2, 0]
            indexer[i_shape - 1, j_shape - 1, 0] += structure_3d[1, 1, 1]
            indexer[i_shape - 1, j_shape - 1, 1] += structure_3d[1, 1, 0]
            
        if image[i_shape - 1, j_shape - 1, k_shape - 1]:
            indexer[i_shape - 2, j_shape - 2, k_shape - 2] += structure_3d[2, 2, 2]
            indexer[i_shape - 2, j_shape - 2, k_shape - 1] += structure_3d[2, 2, 1]
            indexer[i_shape - 2, j_shape - 1, k_shape - 2] += structure_3d[2, 1, 2]
            indexer[i_shape - 2, j_shape - 1, k_shape - 1] += structure_3d[2, 1, 1]
            
            indexer[i_shape - 1, j_shape - 2, k_shape - 2] += structure_3d[1, 2, 2]
            indexer[i_shape - 1, j_shape - 2, k_shape - 1] += structure_3d[1, 2, 1]
            indexer[i_shape - 1, j_shape - 1, k_shape - 2] += structure_3d[1, 1, 2]
            indexer[i_shape - 1, j_shape - 1, k_shape - 1] += structure_3d[1, 1, 1]
        
        #
        # Do the edges
        #
        for k in range(1, k_shape - 1):
            if image[0, 0, k]:
                indexer[0, 0, k - 1] += structure_3d[1, 1, 2]
                indexer[0, 0, k] += structure_3d[1, 1, 1]
                indexer[0, 0, k + 1] += structure_3d[1, 1, 0]
                indexer[0, 1, k - 1] += structure_3d[1, 0, 2]
                indexer[0, 1, k] += structure_3d[1, 0, 1]
                indexer[0, 1, k + 1] += structure_3d[1, 0, 0]
                indexer[1, 0, k - 1] += structure_3d[0, 1, 2]
                indexer[1, 0, k] += structure_3d[0, 1, 1]
                indexer[1, 0, k + 1] += structure_3d[0, 1, 0]
                indexer[1, 1, k - 1] += structure_3d[0, 0, 2]
                indexer[1, 1, k] += structure_3d[0, 0, 1]
                indexer[1, 1, k + 1] += structure_3d[0, 0, 0]
            
            if image[0, j_shape - 1, k]:
                indexer[0, j_shape - 2, k - 1] += structure_3d[1, 2, 2]
                indexer[0, j_shape - 2, k] += structure_3d[1, 2, 1]
                indexer[0, j_shape - 2, k + 1] += structure_3d[1, 2, 0]
                indexer[0, j_shape - 1, k - 1] += structure_3d[1, 1, 2]
                indexer[0, j_shape - 1, k] += structure_3d[1, 1, 1]
                indexer[0, j_shape - 1, k + 1] += structure_3d[1, 1, 0]
                indexer[1, j_shape - 2, k - 1] += structure_3d[0, 2, 2]
                indexer[1, j_shape - 2, k] += structure_3d[0, 2, 1]
                indexer[1, j_shape - 2, k + 1] += structure_3d[0, 2, 0]
                indexer[1, j_shape - 1, k - 1] += structure_3d[0, 1, 2]
                indexer[1, j_shape - 1, k] += structure_3d[0, 1, 1]
                indexer[1, j_shape - 1, k + 1] += structure_3d[0, 1, 0]
            ###
            if image[i_shape - 1, 0, k]:
                indexer[i_shape - 2, 0, k - 1] += structure_3d[2, 1, 2]
                indexer[i_shape - 2, 0, k] += structure_3d[2, 1, 1]
                indexer[i_shape - 2, 0, k + 1] += structure_3d[2, 1, 0]
                indexer[i_shape - 2, 1, k - 1] += structure_3d[2, 0, 2]
                indexer[i_shape - 2, 1, k] += structure_3d[2, 0, 1]
                indexer[i_shape - 2, 1, k + 1] += structure_3d[2, 0, 0]
                indexer[i_shape - 1, 0, k - 1] += structure_3d[1, 1, 2]
                indexer[i_shape - 1, 0, k] += structure_3d[1, 1, 1]
                indexer[i_shape - 1, 0, k + 1] += structure_3d[1, 1, 0]
                indexer[i_shape - 1, 1, k - 1] += structure_3d[1, 0, 2]
                indexer[i_shape - 1, 1, k] += structure_3d[1, 0, 1]
                indexer[i_shape - 1, 1, k + 1] += structure_3d[1, 0, 0]
            
            if image[i_shape - 1, j_shape - 1, k]:
                indexer[i_shape - 2, j_shape - 2, k - 1] += structure_3d[2, 2, 2]
                indexer[i_shape - 2, j_shape - 2, k] += structure_3d[2, 2, 1]
                indexer[i_shape - 2, j_shape - 2, k + 1] += structure_3d[2, 2, 0]
                indexer[i_shape - 2, j_shape - 1, k - 1] += structure_3d[2, 1, 2]
                indexer[i_shape - 2, j_shape - 1, k] += structure_3d[2, 1, 1]
                indexer[i_shape - 2, j_shape - 1, k + 1] += structure_3d[2, 1, 0]
                indexer[i_shape - 1, j_shape - 2, k - 1] += structure_3d[1, 2, 2]
                indexer[i_shape - 1, j_shape - 2, k] += structure_3d[1, 2, 1]
                indexer[i_shape - 1, j_shape - 2, k + 1] += structure_3d[1, 2, 0]
                indexer[i_shape - 1, j_shape - 1, k - 1] += structure_3d[1, 1, 2]
                indexer[i_shape - 1, j_shape - 1, k] += structure_3d[1, 1, 1]
                indexer[i_shape - 1, j_shape - 1, k + 1] += structure_3d[1, 1, 0]
                
        for j in range(1, j_shape - 1):
            if image[0, j, 0]:
                indexer[0, j - 1, 0] += structure_3d[1, 2, 1]
                indexer[0, j - 1, 1] += structure_3d[1, 2, 0]
                indexer[0, j, 0] += structure_3d[1, 1, 1]
                indexer[0, j, 1] += structure_3d[1, 1, 0]
                indexer[0, j + 1, 0] += structure_3d[1, 0, 1]
                indexer[0, j + 1, 1] += structure_3d[1, 0, 0]
                indexer[1, j - 1, 0] += structure_3d[0, 2, 1]
                indexer[1, j - 1, 1] += structure_3d[0, 2, 0]
                indexer[1, j, 0] += structure_3d[0, 1, 1]
                indexer[1, j, 1] += structure_3d[0, 1, 0]
                indexer[1, j + 1, 0] += structure_3d[0, 0, 1]
                indexer[1, j + 1, 1] += structure_3d[0, 0, 0]
            
            if image[0, j, k_shape - 1]:
                indexer[0, j - 1, k_shape - 2] += structure_3d[1, 2, 2]
                indexer[0, j - 1, k_shape - 1] += structure_3d[1, 2, 1]
                indexer[0, j, k_shape - 2] += structure_3d[1, 1, 2]
                indexer[0, j, k_shape - 1] += structure_3d[1, 1, 1]
                indexer[0, j + 1, k_shape - 2] += structure_3d[1, 0, 2]
                indexer[0, j + 1, k_shape - 1] += structure_3d[1, 0, 1]
                indexer[1, j - 1, k_shape - 2] += structure_3d[0, 2, 2]
                indexer[1, j - 1, k_shape - 1] += structure_3d[0, 2, 1]
                indexer[1, j, k_shape - 2] += structure_3d[0, 1, 2]
                indexer[1, j, k_shape - 1] += structure_3d[0, 1, 1]
                indexer[1, j + 1, k_shape - 2] += structure_3d[0, 0, 2]
                indexer[1, j + 1, k_shape - 1] += structure_3d[0, 0, 1]
            ###
            if image[i_shape - 1, j, 0]:
                indexer[i_shape - 2, j - 1, 0] += structure_3d[2, 2, 1]
                indexer[i_shape - 2, j - 1, 1] += structure_3d[2, 2, 0]
                indexer[i_shape - 2, j, 0] += structure_3d[2, 1, 1]
                indexer[i_shape - 2, j, 1] += structure_3d[2, 1, 0]
                indexer[i_shape - 2, j + 1, 0] += structure_3d[2, 0, 1]
                indexer[i_shape - 2, j + 1, 1] += structure_3d[2, 0, 0]
                indexer[i_shape - 1, j - 1, 0] += structure_3d[1, 2, 1]
                indexer[i_shape - 1, j - 1, 1] += structure_3d[1, 2, 0]
                indexer[i_shape - 1, j, 0] += structure_3d[1, 1, 1]
                indexer[i_shape - 1, j, 1] += structure_3d[1, 1, 0]
                indexer[i_shape - 1, j + 1, 0] += structure_3d[1, 0, 1]
                indexer[i_shape - 1, j + 1, 1] += structure_3d[1, 0, 0]
            
            if image[i_shape - 1, j, k_shape - 1]:
                indexer[i_shape - 2, j - 1, k_shape - 2] += structure_3d[2, 2, 2]
                indexer[i_shape - 2, j - 1, k_shape - 1] += structure_3d[2, 2, 1]
                indexer[i_shape - 2, j, k_shape - 2] += structure_3d[2, 1, 2]
                indexer[i_shape - 2, j, k_shape - 1] += structure_3d[2, 1, 1]
                indexer[i_shape - 2, j + 1, k_shape - 2] += structure_3d[2, 0, 2]
                indexer[i_shape - 2, j + 1, k_shape - 1] += structure_3d[2, 0, 1]
                indexer[i_shape - 1, j - 1, k_shape - 2] += structure_3d[1, 2, 2]
                indexer[i_shape - 1, j - 1, k_shape - 1] += structure_3d[1, 2, 1]
                indexer[i_shape - 1, j, k_shape - 2] += structure_3d[1, 1, 2]
                indexer[i_shape - 1, j, k_shape - 1] += structure_3d[1, 1, 1]
                indexer[i_shape - 1, j + 1, k_shape - 2] += structure_3d[1, 0, 2]
                indexer[i_shape - 1, j + 1, k_shape - 1] += structure_3d[1, 0, 1]
                
        for i in range(1, i_shape - 1):
            if image[i, 0, 0]:
                indexer[i - 1, 0, 0] += structure_3d[2, 1, 1]
                indexer[i - 1, 0, 1] += structure_3d[2, 1, 0]
                indexer[i - 1, 1, 0] += structure_3d[2, 0, 1]
                indexer[i - 1, 1, 1] += structure_3d[2, 0, 0]
                indexer[i, 0, 0] += structure_3d[1, 1, 1]
                indexer[i, 0, 1] += structure_3d[1, 1, 0]
                indexer[i, 1, 0] += structure_3d[1, 0, 1]
                indexer[i, 1, 1] += structure_3d[1, 0, 0]
                indexer[i + 1, 0, 0] += structure_3d[0, 1, 1]
                indexer[i + 1, 0, 1] += structure_3d[0, 1, 0]
                indexer[i + 1, 1, 0] += structure_3d[0, 0, 1]
                indexer[i + 1, 1, 1] += structure_3d[0, 0, 0]
            
            if image[i, 0, k_shape - 1]:
                indexer[i - 1, 0, k_shape - 2] += structure_3d[2, 1, 2]
                indexer[i - 1, 0, k_shape - 1] += structure_3d[2, 1, 1]
                indexer[i - 1, 1, k_shape - 2] += structure_3d[2, 0, 2]
                indexer[i - 1, 1, k_shape - 1] += structure_3d[2, 0, 1]
                indexer[i, 0, k_shape - 2] += structure_3d[1, 1, 2]
                indexer[i, 0, k_shape - 1] += structure_3d[1, 1, 1]
                indexer[i, 1, k_shape - 2] += structure_3d[1, 0, 2]
                indexer[i, 1, k_shape - 1] += structure_3d[1, 0, 1]
                indexer[i + 1, 0, k_shape - 2] += structure_3d[0, 1, 2]
                indexer[i + 1, 0, k_shape - 1] += structure_3d[0, 1, 1]
                indexer[i + 1, 1, k_shape - 2] += structure_3d[0, 0, 2]
                indexer[i + 1, 1, k_shape - 1] += structure_3d[0, 0, 1]
            ###
            if image[i, j_shape - 1, 0]:
                indexer[i - 1, j_shape - 2, 0] += structure_3d[2, 2, 1]
                indexer[i - 1, j_shape - 2, 1] += structure_3d[2, 2, 0]
                indexer[i - 1, j_shape - 1, 0] += structure_3d[2, 1, 1]
                indexer[i - 1, j_shape - 1, 1] += structure_3d[2, 1, 0]
                indexer[i, j_shape - 2, 0] += structure_3d[1, 2, 1]
                indexer[i, j_shape - 2, 1] += structure_3d[1, 2, 0]
                indexer[i, j_shape - 1, 0] += structure_3d[1, 1, 1]
                indexer[i, j_shape - 1, 1] += structure_3d[1, 1, 0]
                indexer[i + 1, j_shape - 2, 0] += structure_3d[0, 2, 1]
                indexer[i + 1, j_shape - 2, 1] += structure_3d[0, 2, 0]
                indexer[i + 1, j_shape - 1, 0] += structure_3d[0, 1, 1]
                indexer[i + 1, j_shape - 1, 1] += structure_3d[0, 1, 0]
            
            if image[i, 0, k_shape - 1]:
                indexer[i - 1, j_shape - 2, k_shape - 2] += structure_3d[2, 2, 2]
                indexer[i - 1, j_shape - 2, k_shape - 1] += structure_3d[2, 2, 1]
                indexer[i - 1, j_shape - 1, k_shape - 2] += structure_3d[2, 1, 2]
                indexer[i - 1, j_shape - 1, k_shape - 1] += structure_3d[2, 1, 1]
                indexer[i, j_shape - 2, k_shape - 2] += structure_3d[1, 2, 2]
                indexer[i, j_shape - 2, k_shape - 1] += structure_3d[1, 2, 1]
                indexer[i, j_shape - 1, k_shape - 2] += structure_3d[1, 1, 2]
                indexer[i, j_shape - 1, k_shape - 1] += structure_3d[1, 1, 1]
                indexer[i + 1, j_shape - 2, k_shape - 2] += structure_3d[0, 2, 2]
                indexer[i + 1, j_shape - 2, k_shape - 1] += structure_3d[0, 2, 1]
                indexer[i + 1, j_shape - 1, k_shape - 2] += structure_3d[0, 1, 2]
                indexer[i + 1, j_shape - 1, k_shape - 1] += structure_3d[0, 1, 1]
        #
        # Do the faces
        #
        for k in range(1, k_shape - 1):
            for j in range(1, j_shape - 1):
                if image[0, j, k]:
                    for jj in range(3):
                        for kk in range(3):
                            indexer[1, j - jj + 1, k - kk + 1] += structure_3d[0, jj, kk]
                            indexer[0, j - jj + 1, k - kk + 1] += structure_3d[1, jj, kk]
        for k in range(1, k_shape - 1):
            for j in range(1, j_shape - 1):
                if image[i_shape - 1, j, k]:
                    for jj in range(3):
                        for kk in range(3):
                            indexer[i_shape - 1, j - jj + 1, k - kk + 1] += structure_3d[1, jj, kk]
                            indexer[i_shape - 2, j - jj + 1, k - kk + 1] += structure_3d[2, jj, kk]
        
        for k in range(1, k_shape - 1):
            for i in range(1, i_shape - 1):
                if image[i, 0, k]:
                    for ii in range(3):
                        for kk in range(3):
                            indexer[i - ii + 1, 1, k - kk + 1] += structure_3d[ii, 0, kk]
                            indexer[i - ii + 1, 0, k - kk + 1] += structure_3d[ii, 1, kk]
        for k in range(1, k_shape - 1):
            for i in range(1, i_shape - 1):
                if image[i, j_shape - 1, k]:
                    for ii in range(3):
                        for kk in range(3):
                            indexer[i - ii + 1, j_shape - 1, k - kk + 1] += structure_3d[ii, 1, kk]
                            indexer[i - ii + 1, j_shape - 2, k - kk + 1] += structure_3d[ii, 2, kk]
        
        for j in range(1, j_shape - 1):
            for i in range(1, i_shape - 1):
                if image[0, j, k]:
                    for ii in range(3):
                        for jj in range(3):
                            indexer[i - ii + 1, j - jj + 1, 1] += structure_3d[ii, jj, 0]
                            indexer[i - ii + 1, j - jj + 1, 0] += structure_3d[ii, jj, 1]
        for j in range(1, j_shape - 1):
            for i in range(1, i_shape - 1):
                if image[i_shape - 1, j, k]:
                    for ii in range(3):
                        for jj in range(3):
                            indexer[i - ii + 1, j - jj + 1, k_shape - 1] += structure_3d[ii, jj, 1]
                            indexer[i - ii + 1, j - jj + 1, k_shape - 2] += structure_3d[ii, jj, 2]
                
    return np.asarray(indexer)

def _skeletonize_loop(cnp.uint8_t[:, :, ::1] result,
                      Py_ssize_t[::1] i, Py_ssize_t[::1] j, Py_ssize_t[::1] k,
                      cnp.int32_t[::1] order, cnp.uint8_t[::1] table):
    cdef:
        Py_ssize_t accumulator
        Py_ssize_t index, order_index
        Py_ssize_t ii, jj, kk
        Py_ssize_t rows = result.shape[0]
        Py_ssize_t cols = result.shape[1]
        Py_ssize_t slices = result.shape[2]

    with nogil:
        for index in range(order.shape[0]):
            accumulator = 8192
            order_index = order[index]
            ii = i[order_index]
            jj = j[order_index]
            kk = k[order_index]
            # Compute the configuration around the pixel
            if ii > 0:
                if jj > 0:
                    if kk > 0 and result[ii - 1, jj - 1, kk - 1]:
                        accumulator += 1
                    if result[ii - 1, jj - 1, kk]:
                        accumulator += 2
                    if kk < slices - 1 and result[ii - 1, jj - 1, kk + 1]:
                        accumulator += 4
                if kk > 0 and result[ii - 1, jj, kk - 1]:
                    accumulator += 8
                if result[ii - 1, jj, kk]:
                    accumulator += 16
                if kk < slices - 1 and result[ii - 1, jj, kk + 1]:
                    accumulator += 32
                if jj < cols - 1:
                    if kk > 0 and result[ii - 1, jj + 1, kk - 1]:
                        accumulator += 64
                    if result[ii - 1, jj + 1, kk]:
                        accumulator += 128
                    if kk < slices - 1 and result[ii - 1, jj + 1, kk + 1]:
                        accumulator += 256
                    
            if jj > 0:
                if kk > 0 and result[ii, jj - 1, kk - 1]:
                    accumulator += 512
                if result[ii, jj - 1, kk]:
                    accumulator += 1024
                if kk < slices - 1 and result[ii, jj - 1, kk + 1]:
                    accumulator += 2048
            if kk > 0 and result[ii, jj, kk - 1]:
                accumulator += 4096
            if kk < slices - 1 and result[ii, jj, kk + 1]:
                accumulator += 16384
            if jj < cols - 1:
                if kk > 0 and result[ii, jj + 1, kk - 1]:
                    accumulator += 32768
                if result[ii, jj + 1, kk]:
                    accumulator += 65536
                if kk < slices - 1 and result[ii, jj + 1, kk + 1]:
                    accumulator += 131072
            
            if ii < rows - 1:
                if jj > 0:
                    if kk > 0 and result[ii + 1, jj - 1, kk - 1]:
                        accumulator += 262144
                    if result[ii + 1, jj - 1, kk]:
                        accumulator += 524288
                    if kk < slices - 1 and result[ii + 1, jj - 1, kk + 1]:
                        accumulator += 1048576
                if kk > 0 and result[ii + 1, jj, kk - 1]:
                    accumulator += 2097152
                if result[ii + 1, jj, kk]:
                    accumulator += 4194304
                if kk < slices - 1 and result[ii + 1, jj, kk + 1]:
                    accumulator += 8388608
                if jj < cols - 1:
                    if kk > 0 and result[ii + 1, jj + 1, kk - 1]:
                        accumulator +=16777216
                    if result[ii + 1, jj + 1, kk]:
                        accumulator += 33554432
                    if kk < slices - 1 and result[ii + 1, jj + 1, kk + 1]:
                        accumulator += 67108864
            # Assign the value of table corresponding to the configuration
            result[ii, jj, kk] = table[accumulator]

def fill_Euler_LUT():
    LUT = np.zeros(256, dtype=np.intc)
    cdef:
        int arr[128]

    arr[:] = [1, -1, -1, 1, -3, -1, -1, 1, -1, 1, 1, -1, 3, 1, 1, -1, -3, -1,
                 3, 1, 1, -1, 3, 1, -1, 1, 1, -1, 3, 1, 1, -1, -3, 3, -1, 1, 1,
                 3, -1, 1, -1, 1, 1, -1, 3, 1, 1, -1, 1, 3, 3, 1, 5, 3, 3, 1,
                 -1, 1, 1, -1, 3, 1, 1, -1, -7, -1, -1, 1, -3, -1, -1, 1, -1,
                 1, 1, -1, 3, 1, 1, -1, -3, -1, 3, 1, 1, -1, 3, 1, -1, 1, 1,
                 -1, 3, 1, 1, -1, -3, 3, -1, 1, 1, 3, -1, 1, -1, 1, 1, -1, 3,
                 1, 1, -1, 1, 3, 3, 1, 5, 3, 3, 1, -1, 1, 1, -1, 3, 1, 1, -1]
    LUT[1::2] = arr
    return LUT

def is_Euler_variant(cnp.npy_uint8 neighbors[], int[::1] lut):
    _neighb_idx = [[2, 1, 11, 10, 5, 4, 14],      # NEB
                [0, 9, 3, 12, 1, 10, 4],       # NWB
                [8, 7, 17, 16, 5, 4, 14],      # SEB
                [6, 15, 7, 16, 3, 12, 4],      # SWB
                [20, 23, 19, 22, 11, 14, 10],  # NEU
                [18, 21, 9, 12, 19, 22, 10],   # NWU
                [26, 23, 17, 14, 25, 22, 16],  # SEU
                [24, 25, 15, 16, 21, 22, 12],  # SWU
                ]
    cdef int n, euler_char = 0
    for _octant in range(8):
        n = 1
        for _j in range(7):
            _idx = _neighb_idx[_octant][_j]
            if neighbors[_idx] == 1:
                n |= 1 << (7 - _j)
        euler_char += lut[n]
    return euler_char != 0


import numpy as np
from scipy import ndimage as ndi

def bits_27():
    bits_array = [2**0,2**1,2**2,2**3,2**4,2**5,2**6,2**7,2**8,
                 2**9,2**10,2**11,2**12,2**13,2**14,2**15,2**16,2**17,
                 2**18,2**19,2**20,2**21,2**22,2**23,2**24,2**25,2**26,]
    return bits_array

def _neighbor(index, bits_array):
    cdef: 
        cnp.npy_uint8 neighbor[27]
    for i in range(27):
        neighbor[i] = bool(index & bits_array[i])
    if(index%1000000==0):
        print(index)
    return neighbor

def _pattern_of(index, bits_array):
    if(index%1000000==0):
        print(index)
    return np.array([[[index & bits_array[0], index & bits_array[1], index & bits_array[2]],
                     [index & bits_array[3], index & bits_array[4], index & bits_array[5]],
                     [index & bits_array[6], index & bits_array[7], index & bits_array[8]]],
                    [[index & bits_array[9], index & bits_array[10], index & bits_array[11]],
                     [index & bits_array[12], index & bits_array[13], index & bits_array[14]],
                     [index & bits_array[15], index & bits_array[16], index & bits_array[17]]],
                    [[index & bits_array[18], index & bits_array[19], index & bits_array[20]],
                     [index & bits_array[21], index & bits_array[22], index & bits_array[23]],
                     [index & bits_array[24], index & bits_array[25], index & bits_array[26]]]], bool)

def initialize_LUT():
    print(0)
    _twenty_six_connect = ndi.generate_binary_structure(3, 3)
    _bits_27 = bits_27()
    print(1)
    table = np.array([ndi.label(_pattern_of(index, _bits_27),_twenty_six_connect)[1] !=
                      ndi.label(_pattern_of(index & ~ _bits_27[13], _bits_27),_twenty_six_connect)[1]
                        for index in range(134217728)])
    print(2)
    table = table | np.array([np.sum(_pattern_of(index, _bits_27)) < 3 for index in range(134217728)])
    print(3)
    table = table | np.array([is_Euler_variant(_neighbor(index, _bits_27),fill_Euler_LUT())
                       for index in range(134217728)])
    print(4)
    table = table & (np.arange(134217728) & 2**13).astype(bool)
    

    with open('LUT/medial_axis_3d_LUT.npy', 'wb') as f:
        np.save(f,table)
    print('skeletonize LUT generated')
    
    cornerness_table = np.array([np.sum(_pattern_of(index, _bits_27))
                                 for index in range(134217728)])
    with open('LUT/cornerness_LUT.npy', 'wb') as f:
        np.save(f,cornerness_table)
    print('cornerness LUT generated')

def medial_axis_3d(image, mask=None, return_distance=False, *, random_state=None):
    if mask is None:
        masked_image = image.astype(bool)
    else:
        masked_image = image.astype(bool).copy()
        masked_image[~mask] = False

    # Build distance transform
    distance = ndi.distance_transform_edt(masked_image)
    if return_distance:
        store_distance = distance.copy()

    # Corners
    cornerness_table = np.load('LUT/cornerness_LUT.npy')
    corner_score = _table_lookup_3d(masked_image, np.flip(cornerness_table))

    # Define arrays for inner loop
    i, j, k = np.mgrid[0:image.shape[0], 0:image.shape[1], 0:image.shape[2]]
    result = masked_image.copy()
    distance = distance[result]
    i = np.ascontiguousarray(i[result], dtype=np.intp)
    j = np.ascontiguousarray(j[result], dtype=np.intp)
    k = np.ascontiguousarray(k[result], dtype=np.intp)
    result = np.ascontiguousarray(result, np.uint8)

    # Determine the order in which pixels are processed.

    generator = np.random.default_rng(random_state)
    tiebreaker = generator.permutation(np.arange(masked_image.sum()))
    order = np.lexsort((tiebreaker,
                        corner_score[masked_image],
                        distance))
    order = np.ascontiguousarray(order, dtype=np.int32)
    table = np.load('LUT/medial_axis_3d_LUT.npy')
    table = np.ascontiguousarray(table, dtype=np.uint8)
    # Remove pixels not belonging to the medial axis
    _skeletonize_loop(result, i, j, k, order, table)

    result = result.astype(bool)
    if mask is not None:
        result[~mask] = image[~mask]
    if return_distance:
        return result, store_distance
    else:
        return result




In [7]:
initialize_LUT()

0
1
0
0
0
1000000
1000000
1000000
2000000
2000000
2000000
3000000
3000000
3000000
4000000
4000000
4000000
5000000
5000000
5000000
6000000
6000000
6000000
7000000
7000000
7000000
8000000
8000000
8000000
9000000
9000000
9000000
10000000
10000000
10000000
11000000
11000000
11000000
12000000
12000000
12000000
13000000
13000000
13000000
14000000
14000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
29000000
29000000
30000000
30000000
30000000
31000000
31000000
31000000
32000000
32000000
32000000
33000000
33000000
33000000
34000000
34000000
34000000
35000000
35000000
35000000
36000000
36000000
36000000
37000000
37000000
37000000
38000000
38000000
38000000
39000000
39000000
39000000
40000000
40000000
40000000
41000000
41000000
41000000
42000000
42000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000


In [3]:
from PIL import Image
import numpy as np
import skimage
import skimage.morphology
import skimage.feature
import skan
from scipy import ndimage as ndi
from tifffile import imsave
from scipy.interpolate import interp1d


def generate_sphere(radius, dtype=np.uint8):
    L = np.arange(-radius, radius + 1)
    X, Y, Z = np.meshgrid(L, L, L)
    return np.array((X ** 2 + Y ** 2 + Z ** 2) <= radius ** 2, dtype=dtype)


def save_image_as_tiff(image,filename):
	image = np.einsum('ijk->kij', image)
	image = np.flip(image, axis=(1, 2))
	imsave(filename,image)


def tubularity_detection(image_array, sigma):
	hessian = skimage.feature.hessian_matrix(image_array, sigma=sigma)
	hessian_eig = np.array(skimage.feature.hessian_matrix_eigvals(hessian))
	ra = np.absolute(hessian_eig[1])/np.absolute(hessian_eig[0])
	rb = np.absolute(hessian_eig[2])/np.sqrt(np.absolute(np.multiply(hessian_eig[1], hessian_eig[0])))
	s = np.sqrt(np.square(hessian_eig[1])+np.square(hessian_eig[2]))
	c = (np.absolute(hessian[0])+np.absolute(hessian[1])+np.absolute(hessian[2])+np.absolute(hessian[3])+
	np.absolute(hessian[4])+np.absolute(hessian[5]))/2
	c = np.square(c)*2
	tubularity = np.multiply(1-np.exp(-(np.square(ra)/0.5)), np.exp(-(np.square(rb)/0.5)))
	tubularity = np.multiply(tubularity, 1-np.exp(-np.divide(np.square(s), c)))
	return tubularity


def image_pro(image_array):
	image_array = image_array / np.amax(image_array)
	image_array = skimage.filters.gaussian(image_array, sigma=1)
	image_array = skimage.morphology.closing(image_array)

	block_size = 9
	#threshold = np.zeros(image_array.shape)
	#for j in range(image_array.shape[2]):
	#	threshold[:, :, j] = skimage.filters.threshold_local(image_array[:, :, j], block_size, offset=0)
	mask = image_array > 0.15
	mask = mask | remove_hole(~mask)
	#mask = skimage.morphology.closing(mask,generate_sphere(3))
	mask = ndi.binary_fill_holes(mask)
	'''
	sigma_range = 29
	x, y, z = image_array.shapeinter
	tubularity = np.stack([mask, mask], axis=3)
	print tubularity.shape
	for j in range(sigma_range):
		tubularity[:, :, :, 1] = tubularity_detection(mask, i+1)
		tubularity[:, :, :, 0] = np.amax(tubularity,axis=3)
	return tubularity[:, :, :, 0]
	'''
	return mask


def remove_false_hole(binary_voxel_image):
	labelled_image = skimage.measure.label(binary_voxel_image)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_bbox_area = [objf["bbox_area"] for objf in object_features]
	object_bbox = [objf["bbox"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	small_object = np.array([[], [], []])
	small_object = np.einsum('ij->ji', small_object)
	for j in range(len(object_bbox)):
		bbox = object_bbox[j]
		if ((bbox[5]-bbox[2]) < 5) | (object_area[j] < object_bbox_area[j]/6):
			small_object = np.concatenate([small_object, object_coordinate[j]])
	small_object = np.array(small_object, dtype=int)
	l, ll = small_object.shape
	for j in range(l):
		binary_voxel_image[small_object[j, 0], small_object[j, 1], small_object[j, 2]] = False
	return binary_voxel_image


def remove_hole(binary_voxel_image):
	fill = np.zeros(binary_voxel_image.shape, dtype=bool)
	for k in range(binary_voxel_image.shape[2]):
		binary_image = binary_voxel_image[:, :, k]
		labelled_image = skimage.measure.label(binary_image)
		object_features = skimage.measure.regionprops(labelled_image)
		object_area = [objf["area"] for objf in object_features]
		object_coordinate = [objf["coords"] for objf in object_features]
		small_object = np.array([[], []])
		small_object = np.einsum('ij->ji', small_object)
		for j in range(len(object_area)):
			if object_area[j] < 20000:
				small_object = np.concatenate([small_object, object_coordinate[j]])
		small_object = np.array(small_object, dtype=int)
		l, ll = small_object.shape
		for j in range(l):
			fill[small_object[j, 0], small_object[j, 1], k] = True
	return remove_false_hole(fill)


def remove_hole2(binary_voxel_image):
	fill = np.zeros(binary_voxel_image.shape, dtype=bool)
	labelled_image = skimage.measure.label(binary_voxel_image)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	if len(object_area) > 1:
		object_coordinate.pop(object_area.index(max(object_area)))
		if len(object_area) > 2:
			small_object = np.concatenate(object_coordinate, axis=0)
		else:
			small_object = object_coordinate[0]
		print (small_object.shape)
		l, ll = small_object.shape
		for j in range(l):
			fill[small_object[j, 0], small_object[j, 1], small_object[j, 2]] = True
	return fill


def obtain_skeleton(binary_voxel_image):
	skeleton = medial_axis_3d(binary_voxel_image)
	labelled_image = skimage.measure.label(skeleton)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	max_skeleton = np.zeros(skeleton.shape)
	max_skeleton_coordinate = object_coordinate[object_area.index(max(object_area))]
	l, ll = max_skeleton_coordinate.shape
	for j in range(l):
		max_skeleton[max_skeleton_coordinate[j, 0], max_skeleton_coordinate[j, 1], max_skeleton_coordinate[j, 2]] = 1
		max_skeleton[max_skeleton_coordinate[j, 0], max_skeleton_coordinate[j, 1], max_skeleton_coordinate[j, 2]] = 1
	with open('np arrays/skeleton.npy', 'wb') as f:
		np.save(f,max_skeleton)
	return max_skeleton


def pruning(binary_voxel_image):
	x, y, z = np.shape(binary_voxel_image)
	original_image = binary_voxel_image
	binary_voxel_image = np.array(dtype=uint8)
	binary_voxel_image = np.pad(binary_voxel_image, (1, 1), mode='constant', constant_values=1)
	neighbor=binary_voxel_image_padded[:-2, 1:-1, 1:-1] + binary_voxel_image[:-2, :-2, 1:-1] + binary_voxel_image[:-2, 2:, 1:-1] \
	         + binary_voxel_image[:-2, 1:-1, 2:] + binary_voxel_image[:-2, :-2, 2:] + binary_voxel_image[:-2, 2:, 2:] \
	         + binary_voxel_image[:-2, 1:-1, :-2] + binary_voxel_image[:-2, :-2, :-2] + binary_voxel_image[:-2, 2:, :-2] \
	         + binary_voxel_image[2:, 1:-1, :-2] + binary_voxel_image[2:, :-2, :-2] + binary_voxel_image[2:, 2:, :-2] \
	         + binary_voxel_image[2:, 1:-1, 1:-1] + binary_voxel_image[2:, :-2, 1:-1] + binary_voxel_image[2:, 2:, 1:-1] \
	         + binary_voxel_image[2:, 1:-1, 2:] + binary_voxel_image[2:, :-2, 2:] + binary_voxel_image[2:, 2:, 2:] \
	         + binary_voxel_image[1:-1, 1:-1, :-2] + binary_voxel_image[1:-1, :-2, :-2] + binary_voxel_image[1:-1, 2:, :-2] \
	         + binary_voxel_image[1:-1, :-2, 1:-1] + binary_voxel_image[1:-1, 2:, 1:-1] \
	         + binary_voxel_image[1:-1, 1:-1, 2:] + binary_voxel_image[1:-1, :-2, 2:] + binary_voxel_image[1:-1, 2:, 2:] \

	def trace_dendrite(x,y,z):
		while neighbor<=2:
			a=1
	for i in range(x-1):
		for j in range(y-1):
			for k in range(z-1):
				if original_image[i, j, k] & neighbor[x,y,z]==1:
					trace_dendrite()
				break

In [4]:
img = Image.open('nTracer sample.tif')
h, w = np.shape(img)
nframes = int(img.n_frames/4)
img_r = np.zeros((h, w, nframes))
img_g = np.zeros((h, w, nframes))
img_b = np.zeros((h, w, nframes))
img_a = np.zeros((h, w, nframes))
for i in range(nframes - 1):
	img.seek(i*4+1)
	img_r[:, :, i] = np.array(img)
	img.seek(i * 4 + 2)
	img_g[:, :, i] = np.array(img)
	img.seek(i * 4 + 3)
	img_b[:, :, i] = np.array(img)
	img.seek(i * 4 + 4)
	img_a[:, :, i] = np.array(img)
img_rgba = np.max(np.stack([img_r, img_g, img_b, img_a]), axis=0)
f_interpolation = interp1d(np.linspace(0, nframes-1, nframes), img_rgba, axis=2)
img_rgba = f_interpolation(np.linspace(0, nframes-1, 2*nframes-1))

new_image = image_pro(img_rgba)
new_image.astype(np.int8)
new_image = new_image*225
save_image_as_tiff(new_image,'thresholding.tif')
with open('thresholding.npy', 'wb') as f:
	np.save(f,new_image)

FileNotFoundError: [Errno 2] No such file or directory: 'nTracer sample.tif'

In [4]:
image = np.load('np arrays/Layer1_Segmentation_Mask.npy') #dendrites

def find_largest_object(image):
    labelled_array,nFeature = ndi.label(image)
    LargestN = 0
    index = 0
    for i in range(nFeature):
        temp = labelled_array==(i+1)
        if LargestN<np.sum(temp):
            LargestN = np.sum(temp)
            index = i + 1
    print(i)
    return labelled_array==index

soma = find_largest_object(image[0,:,:,:])


811


In [5]:
soma = ndi.binary_erosion(ndi.binary_fill_holes(ndi.binary_dilation(soma,structure = generate_sphere(7))),structure = generate_sphere(3))
np.save('np arrays/soma_mask', soma)
np.save('np arrays/soma_skeleton', skimage.morphology.skeletonize_3d(soma))
imsave('images/soma_mask.tif',np.einsum('ijk->kij', soma.astype(float)*225))

In [6]:
foreground = ndi.binary_fill_holes(np.logical_or(image[0,:,:,:],image[1,:,:,:]))
np.save('np arrays/foreground', foreground)
skeleton0 = skimage.morphology.skeletonize_3d(foreground)
skeleton0 = np.logical_and(np.logical_not(soma),skeleton0)
np.save('np arrays/skeleton_image', skeleton0)
imsave('images/skeleton.tif',np.einsum('ijk->kij', skeleton0.astype(float)*225))